# 3. LME銅3M先物 - ボラティリティ分析

## 概要
LME銅3M先物のボラティリティ特性を包括的に分析します。

### 分析内容
1. ボラティリティサーフェス分析
2. GARCHモデルでのボラティリティモデリング
3. ボラティリティクラスタリング
4. リスクメトリクス計算
5. ボラティリティスマイル分析
6. ボラティリティスピルオーバー効果

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy import stats
from arch import arch_model
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# データベース接続
engine = create_engine('postgresql://Yusuke@localhost:5432/lme_copper_db')

print("=== LME銅3M先物ボラティリティ分析システム ===")
print("データベース接続完了")

## 1. データ読み込みと前処理

In [ ]:
# LME銅3M先物データの読み込み
query = """
SELECT 
    date,
    m3_price as price,
    m3_volume as volume
FROM lme_copper_futures
WHERE m3_price IS NOT NULL
ORDER BY date
"""

df = pd.read_sql_query(query, engine)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# リターン
df['returns'] = df['price'].pct_change()
df['log_returns'] = np.log(df['price'] / df['price'].shift(1))
df['squared_returns'] = df['returns'] ** 2
df['abs_returns'] = df['returns'].abs()

# 欠損値除去
df = df.dropna()

print(f"データ期間: {df.index.min()} - {df.index.max()}")
print(f"総レコード数: {len(df):,}")

# 基本統計
print("\n=== リターン統計 ===")
print(f"平均日次リターン: {df['returns'].mean():.6f}")
print(f"日次ボラティリティ: {df['returns'].std():.6f}")
print(f"年率ボラティリティ: {df['returns'].std() * np.sqrt(252):.4f}")
print(f"最大ドローダウン: {df['returns'].min():.4f}")
print(f"最大アップ: {df['returns'].max():.4f}")